# Introduction to metagenomics analysis

BI practicum 7

In this project we will be using samples from dental calculus, and will compare them with the samples from the ancient tooth roots.




## Part 1

### 1.1. Data

I uploaded raw data on the [Figshare repository](https://figshare.com/articles/_Dead_man_s_teeth_dataset/12152040), along with some additional files.


### 1.2. Analysis Pipeline

#### 1.2.1. Amplicon sequencing.


The output of the sequencing machine is a lot of fragments, each corresponding to a different molecule from the different species you are studying. Some of them are more abundant, some less; we don't know which yet, but now we're going to try to separate these fragments into several piles and try to show that sets of similar fragments originate from the same organism.

DNA sequences can be conservative: the same sequence can be found without too many changes in several species. That's why we cannot tell in advance whether a certain sequence we’ve amplified corresponds to a particular species or genus. For a particularly slow-evolving gene, we may not even be able to tell if it's in the same family! In this case, we can call each of these piles an “Operational Taxonomical Unit” or OTU, and then try to attribute it to one branch on the Tree of Life.


##### 1.2.1A. Qiime2-based


- install qiime2
```
conda install qiime2::qiime2
```

We have single-end reads in FASTQ format with Phred33 encoding.

- importing data
```
qiime tools import \
    --type 'SampleData[SequencesWithQuality]' \
    --input-path data/manifest.tsv \
    --output-path data/sequences.qza \
    --input-format SingleEndFastqManifestPhred33V2
```

*Which didn't work as qiime: command not found.* 

Thus the [archive](https://drive.google.com/file/d/1SIF8T6uaD-kYIQLrOZhYofRkNUD-I920/view?usp=share_link) was downloaded

- Demultiplexing and QC

Our reads are already demultiplexed (1 sample per file). Now it is useful to explore how many sequences were obtained per sample, and to get a summary of the distribution of sequence qualities.

```
qiime demux summarize \
    --i-data data/sequences.qza \
    --o-visualization data/sequences.qzv
```

[Archive](https://drive.google.com/file/d/12j6nHQSCaUipj7SBGfqSJyN9CTP047qW/view?usp=share_link)

- Visualize any .qzv file at https://view.qiime2.org/

![hist](img/fwd_QC_hist.PNG)

![interractive](img/interractive_qc.PNG)

-  Feature table construction (and more QC)

We need to strip a barcode and primer sequences out, and filter chimeric sequences. Here we will use the DADA2 pipeline, as follows:
```
qiime dada2 denoise-single \
    --i-demultiplexed-seqs data/sequences.qza \
    --p-trim-left 35 \
    --p-trunc-len 140 \
    --o-representative-sequences data/rep-seqs.qza \
    --o-table data/table.qza \
    --o-denoising-stats data/stats.qza
```

output: [1](https://drive.google.com/file/d/1koKfoHp-Au9nmd0UcyJwR9FnozrTD0WA/view?usp=share_link), [2](https://drive.google.com/file/d/1DCKyHO6vQqMfsFENa9WH58w9wFEjrS31/view?usp=share_link) , [3](https://drive.google.com/file/d/1mBro_JC3-4dDoUzifVdNi4NzjTenK2u_/view?usp=share_link)

Here we need to select value m for --p-trim-left as a total length of the artificial sequences (barcodes), and value n for  --p-trunc-len  according to Interactive Quality Plot tab in the sequences.qzv (it’s going to be a total length of the sequence after the trimming). 
In our case we know that the primer + adapter length is about 35 bp, and amplicon size is about 145 bp. m = 35 and n = 140 can be a reasonable choice in this case.

Check how many reads are passed the filter and were clustered:
```
qiime metadata tabulate \
    --m-input-file data/stats.qza \
    --o-visualization data/stats.qzv
```

[output](https://drive.google.com/file/d/1X5g5HbFXjBqBFPZRCrge2Iq2s10oam3K/view?usp=share_link) 

![filtered result](img/filtered.PNG)

- FeatureTable and FeatureData summaries

One of the main results of the DADA2 step is a clustering into an amplicon sequence variant (ASV) - a higher-resolution analogue of the traditional Operational Taxonomical Units.  Thus, the Feature Table we just obtained is an equivalent of the OTU tables in other metagenomic pipelines. 

A feature is essentially any unit of observation, e.g., an OTU, a sequence variant, a gene, a metabolite, etc, and a feature table is a matrix of sample X feature abundances (the number of times each feature was observed in each sample). In this case we can think of features as OTUs.

First, let’s create visual summaries of the data - how many sequences are associated with each sample and with each feature, etc.

```
qiime feature-table summarize \
    --i-table data/table.qza \
    --o-visualization data/table.qzv \
    --m-sample-metadata-file data/sample-metadata.tsv
```

[output](https://drive.google.com/file/d/1FxPbOU8V0kDpKwxQ-VYWjIfPd0fxzyjo/view?usp=share_link)

Then we can map feature IDs to sequences, to use these representative sequences in other applications, e.g. BLAST each sequence against the NCBI nt database.
```
qiime feature-table tabulate-seqs \
    --i-data data/rep-seqs.qza \
    --o-visualization data/rep-seqs.qzv
```

[output](https://drive.google.com/file/d/1XuG-ljhUQP7w8tHz23OzeLX6KCNG6pBq/view?usp=share_link)

A file with fastas which are 105bp length each

- Taxonomic analysis

Now we can compare the representative sequences with the taxonomy database, and get the answer to our first question: who lives here?

We need the [database](https://docs.qiime2.org/2024.2/data-resources/#silva-16s-18s-rrna) itself
Database itself is just a fasta file of the 16S representatives, and QIIME2 uses Naive Bayes classifiers. [Here](https://disk.yandex.ru/d/QxQWKV8x5ucxvw) is the pretrained classifier. 

```
qiime feature-classifier classify-sklearn \
    --i-classifier data/silva-138-99-nb-classifier.qza \
    --i-reads data/rep-seqs.qza \
    --o-classification data/taxonomy.qza
```

[output](https://drive.google.com/file/d/1x_Ydxf55LarLzr8xUor3zOQU-_u8_DKI/view?usp=share_link)

And visualise:
```
qiime metadata tabulate \
    --m-input-file data/taxonomy.qza \
    --o-visualization data/taxonomy.qzv
```

[output](https://drive.google.com/file/d/1tYyYy_eOmIxXs-AWakRh4hp-CAxktADY/view?usp=share_link)


Next, we can view the taxonomic composition of our samples with interactive bar plots. Generate those plots with the following command and then open the visualization.

```
qiime taxa barplot \
  --i-table data/table.qza \
  --i-taxonomy data/taxonomy.qza \
  --m-metadata-file data/sample-metadata.tsv \
  --o-visualization data/taxa-bar-plots.qzv
```

[output](https://drive.google.com/file/d/1Q_c7AdTuai9VHMNWCSDLhbJkFBBTdJ2W/view?usp=share_link)

This is a second level of taxonomy

![taxonomy](img/bars_lvl2.PNG)

To further analize the data exporting the data to an ASV table and taxonomy file, and making a few adjustments to the metadata file are needed to be done. To avoid the formatting hassle, it is better to use BIOM binary format. We only need a few commands:

*at this point I figured out the installation of qiime to conda*

* Export ASV table to biom file:

```
qiime tools export --input-path data/table.qza --output-path data/export_biom
```

* Export taxonomy  table:

```
qiime tools export --input-path  data/taxonomy.qza --output-path data/export_biom
```

* Add taxonomy information to biom file:

```
biom add-metadata -i data/export_biom/feature-table.biom -o data/export_biom/feature-table-with-taxonomy.biom --observation-metadata-fp data/export_biom/taxonomy.tsv --sc-separated taxonomy --observation-header OTUID,taxonomy,confidence
```

* Fix header of the metadata file:
```
sed  's/SampleID/NAME/g' data/sample-metadata.tsv > data/sample-metadata.txt
```

A powerful online tool - [MicrobiomeAnalyst](https://www.microbiomeanalyst.ca/MicrobiomeAnalyst/ModuleView.xhtml)

##### 1.2.1B. DADA2-based


Alternatively we can use R packages. See teeth.Rmd

### 1.3. Result Interpretation


We are looking for bacteria from the "red complex": Porphyromonas gingivalis, Tannerella forsythia, Treponema denticola which are responsible for causing severe forms of periodontal disease.

Rarefaction curves

![rarefaction curve](img/rarefaction_curve_1.png)

Filtered abundance view

![abundance](img/families_paradont.PNG)

On this picture samples are organized by peridontal disease and only families from the "red complex" are shown. Samples with the disease do have all 3 families of microorganisms while samples with out it have only 2. Althogh niether of these families are shown on the top-10 taxa which indicates their relatively small abundance. 


## Part 2. Shotgun sequencing

Well, now we will be analyzing shotgun sequencing (whole genome) data. We will try to obtain the ancient sequence of these pathogens and compare them with modern one.

We realized that 1000 years ago periodontal disease was caused by the same bacteria that we can find now in our mouth. But we know that bacteria evolve very quickly, and we have a unique opportunity to explore how it happens.

To investigate it, an affected individual G12 was selected for a dental calculus whole metagenome shotgun sequencing, and reads were assembled into contigs. Metagenome assembly is based on the same principles as for prokaryotic assemblies - usually it is a modification of existing algorithms, taking into account coverage (we expect similar coverage for each OTU) and less aggressive error correction (because in metagenomic samples it can be natural variants or close strain).


### 2.1. Data

Assembly results can be downloaded [here](https://www.dropbox.com/s/f5j52tliumt6etm/G12_assembly.fna.gz?dl=0)

#### 2.2. Shotgun sequence data profiling.


We can classify reads to understand the composition of our samples in more detail - bacteria, viruses and eukaryotes as well. There are several tools and approaches for that, we will try kmer-based Kraken2.

Kraken2 is trying to  assign the reads to different taxa based on a reference database of genomes. It works by comparing the k-mers in the reads to the k-mers in the reference database to determine the taxonomic origin of the reads.

The problem with working with kraken is the very large size of the databases.

Precalculated [results](https://drive.google.com/file/d/12GQBxlomcSSMvFI850tcXfU_JUJgCnJX/view?usp=sharing) and [report](https://drive.google.com/file/d/1AkzEXPCGvAnq4MKNMCIwE--r4769_t2-/view?usp=sharing)

These are obtained through

```
conda install kraken2
kraken2-build --standard --db $DBNAME
kraken2 --db /path/to/kraken2_database --output output_file --report report_file input_fastq_file
```
This file contains the final computed organism abundances, listed one clade per line, tab-separated from the clade's percent abundance.


#### 2.3. Visualization of the Kraken results as a Sankey diagram

Use the [Pavian web application](https://fbreitwieser.shinyapps.io/pavian/) to visualise the classification results obtained with Kraken.

Here is the top of reads in the shotgun sequencing. Compared to the MA it looks well defined, there are no 'other' or 'unknown' groups

![pavian_example](img/pavian.PNG)

![sunckey](img/sunckey.PNG)

#### 2.4. Comparison with ancient Tannerella forsythia genome


Our shotgun assembly is still pretty fragmented, so we will have to align our contigs to reference. Downloaded fasta and gff from [here](https://www.ncbi.nlm.nih.gov/nuccore/NC_016610.1)

- index ref
```
cd data/Part2/
bwa index tannerella_ref.fasta
```

- allign 
```
bwa mem tannerella_ref.fasta G12_assembly.fna.gz > tannerella_align.sam
```

- compress sam
```
samtools view -S -b tannerella_align.sam > tannerella_align.bam
```

- Sort and index BAM file
```
samtools sort tannerella_align.bam -o tannerella_align.sorted.bam
samtools index tannerella_align.sorted.bam
```

OR with pipes

```
bwa mem tannerella_ref.fasta G12_assembly.fna.gz | \
samtools view -S -b - | \
samtools sort -o tannerella_align.sorted.bam - && \
samtools index tannerella_align.sorted.bam
```


![igv](img/igv.PNG)

We can see that some of the regions in the modern strain have zero coverage, and probably were obtained during the strain evolution. 

To automatically select these regions, we can use the bedtools package. 

first, turn obtained alignment file (BAM) to BED with bedtools bamtobed
```
bedtools bamtobed -i tannerella_align.sorted.bam > tannerella_align.sorted.bed
```

- then intersect with annotation file (GFF3) using bedtools intersect to keep only new regions in the modern strain (actually, we are doing subtraction, -v)
```
bedtools intersect -a tannerella_ref.gff3 -b tannerella_align.sorted.bed -v > tannerella.intersected.bed
```


In [16]:
def extract_gene_ids(intersected_file):
    gene_ids = set()
    with open(intersected_file, 'r') as file:
        for line in file:
            fields = line.strip().split('\t')
            gff_info = fields[8]
            if 'Protein Homology' in fields:
                for attribute in gff_info.split(';'):
                    if attribute.strip().startswith('product='):
                        gene_id = attribute.strip().split('=')[1]
                        gene_ids.add(gene_id)
    return gene_ids

# Usage
intersected_file = 'data/Part2/tannerella.intersected.bed'
gene_ids = extract_gene_ids(intersected_file)

# Print or save gene IDs
for gene_id in gene_ids:
    print(gene_id)

DUF3873 domain-containing protein
Abi family protein
TetR/AcrR family transcriptional regulator
cell filamentation protein Fic
DUF4372 domain-containing protein
MMPL family transporter
radical SAM protein
lanthionine synthetase C family protein
sigma-54 dependent transcriptional regulator
IS110 family transposase
transposase
lanthionine synthetase LanC family protein
ParA family protein
PcfJ domain-containing protein
DUF4134 domain-containing protein
PH domain-containing protein
galactosyltransferase-related protein
conjugative transposon protein TraN
DUF4974 domain-containing protein
DUF4133 domain-containing protein
Arm DNA-binding domain-containing protein
ATP-binding protein
IS4 family transposase
DUF3408 domain-containing protein
helicase
rhodanese-like domain-containing protein
DUF1896 domain-containing protein
conjugal transfer protein MobA
beta-ketoacyl-ACP synthase III
type VI secretion system tube protein TssD
conjugative transposon protein TraK
four helix bundle protein
AIPR

There are some amount of transposase from different families which could be one of the main factors for evolution in bacteria.